# Github Colone

In [1]:
# ===== Clone GitHub repo =====
import os

REPO_URL = "https://github.com/irenecliu/A2A_Multi_Agent_System.git"
PROJECT_NAME = "A2A_Multi_Agent_System"
PROJECT_DIR = f"/content/{PROJECT_NAME}"

%cd /content

# Clean old copy if exists
if os.path.exists(PROJECT_DIR):
    print("Removing existing project directory...")
    !rm -rf {PROJECT_DIR}

print("Cloning repository...")
!git clone {REPO_URL}

%cd {PROJECT_NAME}

print("Current working directory:")
!pwd
print("Project files:")
!ls

/content
Cloning repository...
Cloning into 'A2A_Multi_Agent_System'...
remote: Enumerating objects: 11357, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11357 (delta 1), reused 3 (delta 0), pack-reused 11349 (from 1)
Receiving objects: 100% (11357/11357), 88.45 MiB | 14.30 MiB/s, done.
Resolving deltas: 100% (3280/3280), done.
Updating files: 100% (10727/10727), done.
/content/A2A_Multi_Agent_System
Current working directory:
/content/A2A_Multi_Agent_System
Project files:
a2a_client_scenarios.py  requirements.txt
a2a_utils.py		 run_a2a_servers.py
adk_agents.py		 run_demo.py
agents			 scenario1_task_allocation.py
data			 scenario2_negotiation_escalation.py
database_setup.py	 scenario3_multi_step_coordination.py
db.py			 test_complex_query.py
db_tools.py		 test_coordinated_query.py
end-to-end-output.txt	 test_escalation_query.py
mcp_client.py		 test_multi_intent_query.py
mcp_server.py		 test_simple_query.py
README.md		 ven

# Dependency Installation

In [2]:
# ===== Install dependencies =====
import os, sys

%cd /content/A2A_Multi_Agent_System

print("Python version:", sys.version)

# 1) requirements.txt
if os.path.exists("requirements.txt"):
    print("Installing dependencies from requirements.txt ...")
    %pip install -r requirements.txt
else:
    print("No requirements.txt found, skipping this part.")

# 2) ADK + A2A + web stack
print("Installing A2A + Google ADK + web stack ...")
%pip install "google-adk" "a2a-sdk[http-server]" nest-asyncio sniffio anyio httpx uvicorn starlette fastapi python-dotenv requests

print("All dependencies installed (if no error above).")


/content/A2A_Multi_Agent_System
Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Installing dependencies from requirements.txt ...
Installing A2A + Google ADK + web stack ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.5/141.5 kB 5.4 MB/s eta 0:00:00
All dependencies installed (if no error above).


# Initialize SQList Database

In [3]:
%cd /content/A2A_Multi_Agent_System

!python database_setup.py

/content/A2A_Multi_Agent_System
Database initialized with 6 customers and 6 tickets at /content/A2A_Multi_Agent_System/data/customer_service.db


# Set up API Key and Start MCP / A2A Servers

In here you will need to upload your own API_Key to google Colab. On the left handside there's a key icon, click that and that's called "Secrets", where could grant access of your own API Key. Enter your google_api_key to "Value", and enter the name "GOOGLE_API_KEY", then run the following cells.

In [4]:
%cd /content/A2A_Multi_Agent_System

import os

try:
    from google.colab import userdata
    api_key = userdata.get("GOOGLE_API_KEY")
except ImportError:
    api_key = None

if not api_key:
    raise RuntimeError("GOOGLE_API_KEY not found. Make sure you added it in Colab Secrets as 'GOOGLE_API_KEY'.")

os.environ["GOOGLE_API_KEY"] = api_key
print("GOOGLE_API_KEY is set in environment.")

/content/A2A_Multi_Agent_System
GOOGLE_API_KEY is set in environment.


In [5]:
%cd /content/A2A_Multi_Agent_System

import subprocess, sys, time, os

mcp_proc = subprocess.Popen(
    [sys.executable, "-u", "mcp_server.py"],
    stdout=open("mcp.log", "a", buffering=1),
    stderr=subprocess.STDOUT,
)

a2a_proc = subprocess.Popen(
    [sys.executable, "-u", "run_a2a_servers.py"],
    stdout=open("a2a.log", "a", buffering=1),
    stderr=subprocess.STDOUT,
)

time.sleep(8)

print("MCP and A2A servers started in background.")
print("MCP PID:", mcp_proc.pid)
print("A2A PID:", a2a_proc.pid)


/content/A2A_Multi_Agent_System
MCP and A2A servers started in background.
MCP PID: 1388
A2A PID: 1389


In [6]:
import os

def snapshot_logs():
    sizes = {}
    for name in ["mcp.log", "a2a.log"]:
        sizes[name] = os.path.getsize(name) if os.path.exists(name) else 0
    return sizes

def show_new_logs(before_sizes):
    for name in ["a2a.log", "mcp.log"]:
        print(f"\n====== {name} (new logs) ======")
        if not os.path.exists(name):
            print("(log file not found)")
            continue
        with open(name, "r") as f:
            f.seek(before_sizes.get(name, 0))
            new = f.read().strip()
            print(new if new else "(no new logs)")


## Test Queries and Scenarios

# Scenario 1: Task Allocation
Query: "I need help with my account, customer ID 12345"

In [7]:
%cd /content/A2A_Multi_Agent_System
before = snapshot_logs()

!python scenario1_task_allocation.py

show_new_logs(before)

/content/A2A_Multi_Agent_System
=== Scenario 1: Task Allocation ===
User: I need help with my account, customer ID 12345
Assistant: Hi Pat,

Thanks for reaching out. I've located your account.

Here are your current details:
- **Name:** Pat Premium
- **Email:** pat.premium@example.com
- **Status:** Active

How can I help you today?


====== a2a.log (new logs) ======
INFO:     127.0.0.1:36042 - "GET /.well-known/agent-card.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:36042 - "POST / HTTP/1.1" 200 OK
/usr/local/lib/python3.12/dist-packages/google/adk/a2a/executor/a2a_agent_executor.py:202: UserWarning: [EXPERIMENTAL] convert_a2a_request_to_agent_run_request: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  run_request = sel

# Scenario 2: Negotiation and Escalation
Query: "I want to cancel my subscription but I'm having billing issues"

In [8]:
%cd /content/A2A_Multi_Agent_System

before = snapshot_logs()

!python scenario2_negotiation_escalation.py

show_new_logs(before)

/content/A2A_Multi_Agent_System
=== Scenario 2: Negotiation/Escalation ===
User: I want to cancel my subscription but I'm having billing issues
Assistant: Of course, I can help with that. To look into your subscription and the billing issues you've mentioned, could you please provide your customer ID?


====== a2a.log (new logs) ======
INFO:     127.0.0.1:40976 - "GET /.well-known/agent-card.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:40976 - "POST / HTTP/1.1" 200 OK
INFO:     127.0.0.1:50936 - "POST / HTTP/1.1" 200 OK
INFO:     127.0.0.1:38486 - "POST / HTTP/1.1" 200 OK

====== mcp.log (new logs) ======
(no new logs)


# Scenario 3: Multi-Step Coordination
Query: "What's the status of all high-priority tickets for premium customers?"

In [9]:
%cd /content/A2A_Multi_Agent_System

before = snapshot_logs()

!python scenario3_multi_step_coordination.py

show_new_logs(before)

/content/A2A_Multi_Agent_System
=== Scenario 3: Multi-Step Coordination ===
User: What's the status of all high-priority tickets for premium customers?
Assistant: Of course! Here is a summary of all high-priority tickets for your premium customers:

**Pat Premium (pat.premium@example.com)**
*   **Ticket ID:** 6
    *   **Issue:** Billing inquiry: duplicate charge detected
    *   **Status:** Open

Let me know if there's anything else I can help you with


====== a2a.log (new logs) ======
INFO:     127.0.0.1:34040 - "GET /.well-known/agent-card.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:34040 - "POST / HTTP/1.1" 200 OK
🛠 [MCP] list_premium_customers: called | payload={'limit': 100}
🛠 [MCP] list_premium_customers: success | payload={'count': 1}
🛠 [MCP] high_priority_tickets_for_customers: called | payload={'customer_ids': [12345]}
🛠 [MCP] high_priority_tickets_for_customers: success | payload={'count': 1}
INFO:     127.0.0.1:52392 - "POST / HTTP/1.1" 200 OK
INFO:     127.0.0.1:40958 - "PO

# Simple Query: "Get customer information for ID 5"
Single agent, straightforward MCP call

In [10]:
%cd /content/A2A_Multi_Agent_System

before = snapshot_logs()

!python test_simple_query.py

show_new_logs(before)

/content/A2A_Multi_Agent_System
=== Simple Query ===
User: Get customer information for ID 5
Assistant: Hello Elena,

Here is your account information:
- **Name**: Elena Novak
- **Email**: elena@example.com
- **Status**: Active
- **Member Since**: December 3, 2025

How can I help you today?


====== a2a.log (new logs) ======
INFO:     127.0.0.1:43410 - "GET /.well-known/agent-card.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:43410 - "POST / HTTP/1.1" 200 OK
🛠 [MCP] get_customer: called | payload={'customer_id': 5}
🛠 [MCP] get_customer: success | payload={'customer': {'id': 5, 'name': 'Elena Novak', 'email': 'elena@example.com', 'phone': '555-5000', 'status': 'active', 'created_at': '2025-12-03 03:38:00', 'updated_at': '2025-12-03 03:38:00'}}
INFO:     127.0.0.1:47622 - "POST / HTTP/1.1" 200 OK
INFO:     127.0.0.1:35218 - "POST / HTTP/1.1" 200 OK

====== mcp.log (new logs) ======
(no new logs)


# Coordinated Query: "I'm customer 12345 and need help upgrading my account"
Multiple agents coordinate: data fetch + support response

In [11]:
%cd /content/A2A_Multi_Agent_System

before = snapshot_logs()

!python test_coordinated_query.py

show_new_logs(before)

/content/A2A_Multi_Agent_System
=== Coordinated Query ===
User: I'm customer 1 and need help upgrading my account
Assistant: Hi Alice,

Thanks for reaching out. I see you're interested in upgrading your account, and I can confirm that you are eligible to upgrade from your current "standard" plan.

You can complete the upgrade process directly from your account settings page. Please let me know if you have any questions or run into any trouble.

Best,
The Support Team


====== a2a.log (new logs) ======
INFO:     127.0.0.1:40246 - "GET /.well-known/agent-card.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:40246 - "POST / HTTP/1.1" 200 OK
🛠 [MCP] get_customer: called | payload={'customer_id': 1}
🛠 [MCP] get_customer: success | payload={'customer': {'id': 1, 'name': 'Alice Carter', 'email': 'alice@example.com', 'phone': '555-1000', 'status': 'active', 'created_at': '2025-12-03 03:38:00', 'updated_at': '2025-12-03 03:38:00'}}
INFO:     127.0.0.1:33232 - "POST / HTTP/1.1" 200 OK
INFO:     127.0.0

# Complex Query: "Show me all active customers who have open tickets"
Requires negotiation between data and support agents

In [12]:
%cd /content/A2A_Multi_Agent_System

before = snapshot_logs()

!python test_complex_query.py

show_new_logs(before)

/content/A2A_Multi_Agent_System
=== Complex Query ===
User: Show me all active customers who have open tickets
Assistant: Of course! Here is a list of all active customers with open support tickets:

*   **Carla Gomez** (carla@example.com)
    *   **Ticket #2:** "App keeps crashing on login" (Status: open, Priority: high)

*   **Derek Yang** (derek@example.com)
    *   **Ticket #3:** "Refund pending for last order" (Status: in\_progress, Priority: medium)

*   **Elena Novak** (elena@example.com)
    *   **Ticket #4:** "Urgent: Payment failed and account locked" (Status: open, Priority: high)

*   **Pat Premium** (pat.premium@example.com)
    *   **Ticket #5:** "Upgrade request to premium plus" (Status: open, Priority: medium)
    *   **Ticket #6:** "Billing inquiry: duplicate charge detected" (Status: open, Priority: high)

Let me know if you need more details on any of these


====== a2a.log (new logs) ======
INFO:     127.0.0.1:51876 - "GET /.well-known/agent-card.json HTTP/1.1" 200 

# Escalation: "I've been charged twice, please refund immediately!"
Router must identify urgency and route appropriately

In [13]:
%cd /content/A2A_Multi_Agent_System

before = snapshot_logs()

!python test_escalation_query.py

show_new_logs(before)

/content/A2A_Multi_Agent_System
=== Escalation ===
User: I've been charged twice, please refund immediately!
Assistant: Hello,

I understand you've been charged twice, and I sincerely apologize for the inconvenience this has caused.

I have created a high-priority ticket (ID: 7) for this issue and have escalated it directly to our billing department. They will investigate the double charge and get back to you with an update within 2–3 business days.

Thank you for your patience.


====== a2a.log (new logs) ======
INFO:     127.0.0.1:46554 - "GET /.well-known/agent-card.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:46554 - "POST / HTTP/1.1" 200 OK
🛠 [MCP] create_ticket: called | payload={'customer_id': 0, 'issue': 'Double charge / refund request', 'priority': 'high', 'status': 'open'}
🛠 [MCP] create_ticket: success | payload={'ticket': {'id': 7, 'customer_id': 0, 'issue': 'Double charge / refund request', 'status': 'open', 'priority': 'high', 'created_at': '2025-12-03 03:42:05'}}
INFO:     

# Multi-Intent: "Update my email to new@email.com and show my ticket history"
Parallel task execution and coordination

In [14]:
%cd /content/A2A_Multi_Agent_System

before = snapshot_logs()

!python test_multi_intent_query.py

show_new_logs(before)

/content/A2A_Multi_Agent_System
=== Multi-Intent ===
User: My customer ID is 4. Update my email to new@email.com and show my ticket history
Assistant: Hello Derek,

I've successfully updated your email address to `new@email.com`.

Here is a summary of your ticket history:

*   **Ticket #3:** "Refund pending for last order" (Status: In Progress)

Please let me know if there's anything else I can help you with


====== a2a.log (new logs) ======
INFO:     127.0.0.1:49198 - "GET /.well-known/agent-card.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:49198 - "POST / HTTP/1.1" 200 OK
🛠 [MCP] update_customer_email: called | payload={'customer_id': 4, 'new_email': 'new@email.com'}
🛠 [MCP] update_customer_email: success | payload={'customer': {'id': 4, 'name': 'Derek Yang', 'email': 'new@email.com', 'phone': '555-4000', 'status': 'active', 'created_at': '2025-12-03 03:38:00', 'updated_at': '2025-12-03 03:42:29'}}
🛠 [MCP] customer_ticket_history: alias->get_customer_history | payload={'customer_id': 4